In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import time
import warnings
from typing import List
from datetime import datetime, timedelta

import json
import arrow
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import (
    classification_report, 
    accuracy_score,
    f1_score,
    confusion_matrix, 
    roc_curve, 
    roc_auc_score, 
    RocCurveDisplay
)
from langchain.agents import initialize_agent, load_tools, Tool
from langchain.agents.react.base import DocstoreExplorer
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationalBufferWindowMemory
from langchain.docstore import Wikipedia
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

from solidchain.utils.audio import notebook_recorder, from_bytes_to_numpy, from_segment_to_numpy, whisper_transribe

In [3]:
warnings.filterwarnings("ignore")

sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

## Testbed for building language models

All examples come from: https://langchain.readthedocs.io/en/latest/getting_started/getting_started.html

In [4]:
%load_ext dotenv
%dotenv ../apps/api/.env
# %env OPENAI_API_KEY

## Basic LLM

In [6]:
llm = OpenAI(
    model_name="text-curie-001", 
    temperature=0.7, 
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)

In [18]:
text = "Is it over?"
llm(text).strip()

'It depends on what you are referring to.'

## ChatGPT

In [16]:
template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""
prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)

chatgpt_chain = LLMChain(
    llm=OpenAI(temperature=0), 
    prompt=prompt, 
    verbose=True, 
    memory=ConversationalBufferWindowMemory(k=2),
)

In [17]:
chatgpt_chain.predict(
    human_input="Imagine you are designing a new website to with HTML and CSS. Output a mockup of the website you would design."
)



> Entering new LLMChain chain...
Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant 

' I would design a website with a modern, clean look. The homepage would feature a large banner image at the top, followed by a navigation bar with links to the different pages. Below the navigation bar, I would include a section for featured content, such as blog posts or featured products. The main content area would feature a grid layout with images and text for each page. Finally, I would include a footer with contact information and social media links.'

In [18]:
chatgpt_chain.predict(
    human_input="Can you show me the code for the website you designed?"
)



> Entering new LLMChain chain...
Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant 

' Sure! Here is the HTML and CSS code for the website I designed:\n\n<html>\n  <head>\n    <title>My Website</title>\n    <link rel="stylesheet" type="text/css" href="style.css">\n  </head>\n  <body>\n    <div class="banner">\n      <img src="banner.jpg" alt="Banner Image">\n    </div>\n    <div class="navigation">\n      <ul>\n        <li><a href="home.html">Home</a></li>\n        <li><a href="about.html">About</a></li>\n        <li><a href="products.html">Products</a></li>\n        <li><a href="contact.html">Contact</a></li>\n      </ul>\n    </div>\n    <div class="featured">\n      <h2>Featured Content</h2>\n      <div class="featured-item">\n        <img src="featured-1.jpg" alt="Featured Item 1">\n        <h3>Featured Item 1</h3'

## ReAct

In [30]:
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, 
    llm=llm, 
    agent="zero-shot-react-description", 
    max_iterations=5,
    verbose=True, 
)

In [32]:
agent.run("What is Sam Altman's net worth divided by his age?")



> Entering new AgentExecutor chain...
 I need to find out Sam Altman's net worth and age.
Action: Search
Action Input: "Sam Altman net worth"
Observation: The estimated net worth of Sam Altman is $250 million. Samuel H. “Sam” Altman is a 36-year-old American best known for being an Angel Investor.
Thought: I need to calculate the net worth divided by age.
Action: Calculator
Action Input: 250,000,000 / 36
Observation: Answer: 6,944,444.44
Thought: I now know the final answer.
Final Answer: Sam Altman's net worth divided by his age is 6,944,444.44.

> Finished chain.


"Sam Altman's net worth divided by his age is 6,944,444.44."

## Voice Input (ASR)

In [5]:
recorder = notebook_recorder()
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [27]:
# audio_semgent = AudioSegment.from_file("test.webm")
audio_semgent = from_bytes_to_numpy(recorder.audio.value)
audio_semgent

array([ 0.0000000e+00, -3.0517578e-05, -3.0517578e-05, ...,
       -6.7138672e-04, -2.1972656e-03, -1.4953613e-03], dtype=float32)

In [28]:
output = whisper_transribe(audio_semgent)
output

{'text': ' Hola, ¿cómo estás?'}